# Introduction

This notebook contains
- Plotting the match statistics
- Visualize gameplay (see the last cell)

Refer to my [discussion post](https://www.kaggle.com/competitions/kore-2022/discussion/320987) for plans and suggestions.

In [ ]:
%%capture

%reset -sf
!pip install --user kaggle-environments > /dev/null
!rm *.py *.pickle

from IPython.core.magic import register_cell_magic

@register_cell_magic
def writefile_and_run(line, cell):
    argz = line.split()
    file = argz[-1]
    mode = 'w'
    if len(argz) == 2 and argz[0] == '-a':
        mode = 'a'
    with open(file, mode) as f:
        f.write(cell)
    get_ipython().run_cell(cell)

In [ ]:
%%writefile_and_run kore_analysis.py

import os, re, json, enum, glob, shutil, collections, requests, pickle

import numpy as np
import matplotlib
import matplotlib.animation
import matplotlib.patheffects
import matplotlib.pyplot as plt
import IPython.display

import kaggle_environments


plt.rcParams["interactive"] = False
plt.rcParams["animation.html"] = "jshtml"
plt.rcParams["animation.embed_limit"] = 70.0   # default 20.0 stopped at step 200
plt.rcParams["figure.figsize"] = [8,8]
plt.rcParams["figure.dpi"] = 100
plt.rcParams["savefig.facecolor"] = "white"


def load_from_simulated_game(home_agent_path, away_agent_path):
    env = kaggle_environments.make("kore_fleets", debug=True)
    env.run([home_agent_path, away_agent_path])
    return env

def load_from_replay_json(json_path):
    with open(path_to_json, 'r') as f:
        match = json.load(f)
    env = kaggle_environments.make("kore_fleets", steps=match['steps'],
                                   configuration=match['configuration'])
    home_agent = "home"
    away_agent = "away"
    return env

def load_from_episode_id(episode_id):
    # kaggle.com/code/robga/kore-episode-scraper-match-downloader/
    base_url = "https://www.kaggle.com/api/i/competitions.EpisodeService/"
    get_url = base_url + "GetEpisodeReplay"
    req = requests.post(get_url, json = {"episodeId": int(episode_id)}).json()
    match = json.loads(req["replay"])
    env = kaggle_environments.make("kore_fleets", steps=match['steps'],
                                   configuration=match['configuration'])
    return env

# Run match or load

In [ ]:
# various ways to load your agent

# simulate a match between two agents
home_agent_path = "../input/kore-beta-1st-place-solution/main.py"
away_agent_path = "../input/kore-starter-6th-in-beta-rule-based-agent/main.py"
# env = load_from_simulated_game(home_agent_path, away_agent_path)

# from match replay file saved
file_pattern = "../input/kore-episode-scraper-match-downloader/*.json"
path_to_json = sorted(fn for fn in glob.glob(file_pattern) if "_" not in fn)[-2]
env = load_from_replay_json(path_to_json)

# from episode_id
episode_id = 36615058
env = load_from_episode_id(episode_id)

In [ ]:
# save env object for animation later
with open('env.pickle', 'wb') as handle:
    pickle.dump(env, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
# object extracted
type(env.steps), len(env.steps)

In [ ]:
env.render(mode="ipython", width=720, height=680)

# Information Extraction

Here we compile the information of the action, shipyards, fleets, and kore storage fo each player.

In [ ]:
%%writefile_and_run -a kore_analysis.py

class FlightPlanClass(enum.IntEnum):
    invalid = 0
    unknown = 1
    acyclic = 2
    boomerang = 3
    rectangle = 4

def kore_mining_rate(kore_amount, fleetsize):
    kore_amount_before_regeneration = kore_amount / 1.02
    if kore_amount_before_regeneration < 500:
        kore_amount = kore_amount_before_regeneration
    precentage_mining_rate = np.log(max(1,fleetsize)) / 20
    kore_amount = kore_amount / (1-precentage_mining_rate)
    return kore_amount * precentage_mining_rate

def calculate_mining_rates(kore_amount_matrices, agent_fleetsize_matrices):
    return [sum(kore_mining_rate(kore_amount, fleetsize) 
                for kore_amount, fleetsize in zip(kore_amounts, fleetsizes))
            for kore_amounts, fleetsizes in zip(kore_amount_matrices, agent_fleetsize_matrices)]

def split_into_number_and_char(srr):
    # https://stackoverflow.com/q/430079/5894029
    arr = []
    for word in re.split('(\d+)', srr):
        try:
            num = int(word)
            arr.append(num)
        except ValueError:
            for c in word:
                arr.append(c)
    return arr

def extract_flight_plan(x,y,dir_idx,plan,endpoints=set()):
    dir_to_dxdy = [(0,1), (1,0), (0,-1), (-1,0)]  # NESW
    dcode_to_dxdy = {"N":(0,1), "E":(1,0), "S":(0,-1), "W":(-1,0)}
    dx,dy = dir_to_dxdy[dir_idx]
    
    plan = collections.deque(split_into_number_and_char(plan))
    
    cx,cy = x, y
    path = [(cx,cy)]
    construct = []
    first_move_complete = False
    
    while plan:
        if first_move_complete and (cx, cy) in endpoints:
            return path, construct, (cx, cy) == (x,y)
        first_move_complete = True
        word = plan.popleft()
        if type(word) == int:
            cx += dx
            cy += dy
            path.append((cx,cy))
            word -= 1
            if word > 0:
                plan.appendleft(word)
            continue
        if word == "C":
            construct.append((cx,cy))
            continue
        dx,dy = dcode_to_dxdy[word]
        cx += dx
        cy += dy
        path.append((cx,cy))
        
    is_cyclic = False
    for _ in range(30):
        if cx == x and cy == y:
            is_cyclic = True
        cx += dx
        cy += dy

    return path, construct, is_cyclic

def fleetplan_matching(flight_plan):
    # plan class, target_x, target_y, polarity, construct
    
    # plan class - boomerang, rectangle, acyclic
    # polarity - whether first move vertical or horizontal
    # whether construct is genuine will not be analyzed here
    
    if not re.match("^[NSEW][NSEWC0-9]*$", flight_plan):
        return (FlightPlanClass.invalid, 0, 0, False, [])

    polarity = (flight_plan[0] == "N") or (flight_plan[0] == "S")

    path, construct, is_cyclic = extract_flight_plan(0,0,0,flight_plan)
    
    x_max_extent = 0
    y_max_extent = 0
    target_x, target_y = 0, 0
    for x,y in path:
        if abs(x) > x_max_extent or abs(y) > y_max_extent:
            x_max_extent = abs(x)
            y_max_extent = abs(y)
            target_x, target_y = x, y
            
    # orbit
    if re.match("^[NSEW]$", flight_plan):
        return (FlightPlanClass.acyclic, target_x, target_y, polarity, construct)

    # sneek peek, yo-yo
    for d1,d2 in zip("NSEW", "SNWE"):
        if re.match(f"^[{d1}][0-9]*[{d2}][0-9]*$", flight_plan):
            return (FlightPlanClass.boomerang, target_x, target_y, polarity, construct)
    
    # travelling
    for d1,d2 in zip("NSEW", "SNWE"):
        if re.match(f"[NSEW][0-9]*[NSEW][0-9]*$", flight_plan):
            return (FlightPlanClass.acyclic, target_x, target_y, polarity, construct)

    # flat rectangle, rectangle
    if is_cyclic:
        for d1,d2 in zip("NSEW", "SNWE"):
            if re.match(f"^[{d1}][0-9]*[NSEW][0-9]*[{d2}][0-9]*[NSEW][0-9]*$", flight_plan):  
                return (FlightPlanClass.rectangle, target_x, target_y, polarity, construct)

    # crowbar, boomerang
    if is_cyclic:
        for d1,d2 in zip("NSEW", "SNWE"):
            if re.match(f"^[NSEW][0-9]*[{d1}][0-9]*[{d2}][0-9]*[NSEW][0-9]*$", flight_plan):  
                return (FlightPlanClass.boomerang, target_x, target_y, polarity, construct)
    
    return (FlightPlanClass.unknown, target_x, target_y, polarity, construct)

class KoreMatch():
    def __init__(self, match_info, home_agent="home", away_agent="away", save_animation=False):
        self.match_info = match_info
        self.home_agent = home_agent
        self.away_agent = away_agent
        self.save_animation = save_animation

        res = match_info
        self.home_actions = [home_info["action"] for home_info,_ in res]
        self.away_actions = [away_info["action"] for _,away_info in res]
        
        self.home_kore_stored = [info[0]["observation"]["players"][0][0] for info in res]
        self.away_kore_stored = [info[0]["observation"]["players"][1][0] for info in res]

        self.home_shipyards = [info[0]["observation"]["players"][0][1] for info in res]
        self.away_shipyards = [info[0]["observation"]["players"][1][1] for info in res]        
        self.all_shipyards_locations = [
            set(kaggle_environments.helpers.Point.from_index(int(loc_idx), 21) for loc_idx, _, _ in home_shipyards.values()) |
            set(kaggle_environments.helpers.Point.from_index(int(loc_idx), 21) for loc_idx, _, _ in away_shipyards.values())
            for home_shipyards, away_shipyards in zip(self.home_shipyards, self.away_shipyards)
         ]
        self.home_fleets = [info[0]["observation"]["players"][0][2] for info in res]
        self.away_fleets = [info[0]["observation"]["players"][1][2] for info in res]
        
        self.home_kore_carried = [sum(x[1] for x in fleet_info.values()) for fleet_info in self.home_fleets]
        self.away_kore_carried = [sum(x[1] for x in fleet_info.values()) for fleet_info in self.away_fleets]
        
        self.home_ship_standby = [sum(shipyard[1] for shipyard in shipyards.values()) for shipyards in self.home_shipyards]
        self.away_ship_standby = [sum(shipyard[1] for shipyard in shipyards.values()) for shipyards in self.away_shipyards]
        self.home_ship_launched = [sum(fleet[2] for fleet in fleets.values()) for fleets in self.home_fleets]
        self.away_ship_launched = [sum(fleet[2] for fleet in fleets.values()) for fleets in self.away_fleets]
        
        self.home_fleetsize_matrices = [[0 for _ in info[0]["observation"]["kore"]] for info in res]
        self.away_fleetsize_matrices = [[0 for _ in info[0]["observation"]["kore"]] for info in res]

        for turn, (home_fleets_info, away_fleets_info) in enumerate(zip(self.home_fleets, self.away_fleets)):
            for home_fleet_info in home_fleets_info.values():
                location, fleetsize = home_fleet_info[0], home_fleet_info[2]
                self.home_fleetsize_matrices[turn][location] += fleetsize
            for away_fleet_info in away_fleets_info.values():
                location, fleetsize = away_fleet_info[0], away_fleet_info[2]
                self.away_fleetsize_matrices[turn][location] += fleetsize
                
        self.kore_amount_matrices = [info[0]["observation"]["kore"] for info in res]

        self.home_mining_rates = calculate_mining_rates(self.kore_amount_matrices, self.home_fleetsize_matrices)
        self.away_mining_rates = calculate_mining_rates(self.kore_amount_matrices, self.away_fleetsize_matrices)

        self.home_spawing_costs = [-10 * sum(int(action.split("_")[1]) for action in actions.values() 
                                        if action.startswith("SPAWN")) for actions in self.home_actions]
        self.away_spawing_costs = [-10 * sum(int(action.split("_")[1]) for action in actions.values() 
                                        if action.startswith("SPAWN")) for actions in self.away_actions]

        self.home_launch_counts = [[int(action.split("_")[1]) for action in actions.values() if action.startswith("LAUNCH")]
                                   for actions in self.home_actions]
        self.away_launch_counts = [[int(action.split("_")[1]) for action in actions.values() if action.startswith("LAUNCH")] 
                                   for actions in self.away_actions]
        self.home_launch_plans = [[(action.split("_")[2]) for action in actions.values() if action.startswith("LAUNCH")]
                                  for actions in self.home_actions]
        self.away_launch_plans = [[(action.split("_")[2]) for action in actions.values() if action.startswith("LAUNCH")] 
                                  for actions in self.away_actions]
        
        self.home_combat_diffs = [(a2+b2-a1-b1)-x-y for x,y,a1,b1,a2,b2 in 
                             zip(self.home_mining_rates[1:], self.home_spawing_costs[1:], self.home_kore_carried, self.home_kore_stored, 
                                 self.home_kore_carried[1:], self.home_kore_stored[1:])]
        self.away_combat_diffs = [(a2+b2-a1-b1)-x-y for x,y,a1,b1,a2,b2 in 
                             zip(self.away_mining_rates[1:], self.away_spawing_costs[1:], self.away_kore_carried, self.away_kore_stored, 
                                 self.away_kore_carried[1:], self.away_kore_stored[1:])]
        
        self.home_kore_asset_sums = 500*np.array(list(map(len,self.home_shipyards))) \
                                   + 10*np.array(self.home_ship_standby) + 10*np.array(self.home_ship_launched) \
                                      + np.array(self.home_kore_stored) + np.array(self.home_kore_carried)
        self.away_kore_asset_sums = 500*np.array(list(map(len,self.away_shipyards))) \
                                   + 10*np.array(self.away_ship_standby) + 10*np.array(self.away_ship_launched) \
                                      + np.array(self.away_kore_stored) + np.array(self.away_kore_carried)
        
    def plot_statistics_kore(self):
        plt.figure(figsize=(15,5))
        plt.plot(self.home_kore_stored, label=self.home_agent + " (stored)", color="blue", linestyle="dotted")
        plt.plot(self.away_kore_stored, label=self.away_agent + " (stored)", color="red", linestyle="dotted")
        plt.plot(self.home_kore_carried, label=self.home_agent + " (carried)", color="blue")
        plt.plot(self.away_kore_carried, label=self.away_agent + " (carried)", color="red")
        plt.title("Kore carried and stored over time")
        plt.xlim(-20,400+20)
        plt.legend()
        plt.show()
        
    def plot_statistics_shipyards(self):
        plt.figure(figsize=(15,4))
        plt.stairs(list(map(len,self.home_shipyards)), label=self.home_agent, lw=1.5, baseline=None, color="blue")
        plt.stairs(list(map(len,self.away_shipyards)), label=self.away_agent, lw=1.5, baseline=None, color="red")
        plt.title("Number of shipyards over time")
        plt.xlim(-20,400+20)
        plt.legend()
        plt.show()
        
    def plot_statistics_ships(self):        
        plt.figure(figsize=(15,5))
        plt.stairs(self.home_ship_standby, label=self.home_agent + " (standby)", baseline=None, color="blue")
        plt.stairs(self.away_ship_standby, label=self.away_agent + " (standby)", baseline=None, color="red")
        plt.stairs(self.home_ship_launched, label=self.home_agent + " (launched)", baseline=None, color="blue", linestyle="dotted")
        plt.stairs(self.away_ship_launched, label=self.away_agent + " (launched)", baseline=None, color="red", linestyle="dotted")
        plt.title("Ships standby and launched over time")
        plt.xlim(-20,400+20)
        plt.legend()
        plt.show()
 
    def plot_statistics_kore_rates(self):        
        plt.figure(figsize=(15,5))
        plt.plot(self.home_mining_rates, label=self.home_agent + " (mining)", color="blue")
        plt.plot(self.away_mining_rates, label=self.away_agent + " (mining)", color="red")
        plt.stairs(self.home_spawing_costs, label=self.home_agent + " (spawning)", baseline=None, color="blue")
        plt.stairs(self.away_spawing_costs, label=self.away_agent + " (spawning)", baseline=None, color="red")
        plt.title("Kore change rates over time")
        plt.xlim(-20,400+20)
        plt.legend()
        plt.show()

    def plot_statistics_combat_diffs(self):
        plt.figure(figsize=(15,5))
        plt.stairs(self.home_combat_diffs, label=self.home_agent + "(combat)", baseline=None, color="blue")
        plt.stairs(self.away_combat_diffs, label=self.away_agent + "(combat)", baseline=None, color="red")
        plt.title("Kore combat diffs over time")
        plt.xlim(-20,400+20)
        plt.legend()
        plt.show()
        
    def plot_statistics_asset_sums(self):        
        plt.figure(figsize=(15,3))
        plt.stairs(self.home_kore_asset_sums, label=self.home_agent, baseline=None, color="blue")
        plt.stairs(self.away_kore_asset_sums, label=self.away_agent, baseline=None, color="red")
        plt.title("Value of assets in terms of Kore over time")
        plt.xlim(-20,400+20)
        plt.legend()
        plt.show()
        
    def plot_statistics_launch_sizes(self):
        display_limit = 100
        limits = [1,2,3,5,8,13,21,34,55]
        
        plt.figure(figsize=(15,6))
        for limit in limits:
            plt.axhline(limit, color="gainsboro", zorder=0)
            plt.axhline(-limit, color="gainsboro", zorder=0)
        
        home_xpts, home_ypts = [], []
        home_xpts_extra, home_ypts_extra = [], []
        for turn_idx, (launch_counts, launch_plans) in enumerate(zip(self.home_launch_counts, self.home_launch_plans)):
            for launch_count, launch_plan in zip(launch_counts, launch_plans):
                _, _, is_cyclic = extract_flight_plan(0,0,0,launch_plan)
                if "C" in launch_plan:
                    home_xpts_extra.append(turn_idx)
                    home_ypts_extra.append(launch_count)
                    continue
                home_xpts.append(turn_idx)
                home_ypts.append(launch_count)
        plt.scatter(home_xpts, home_ypts, color="blue", s=4, label=self.home_agent)
        plt.scatter(home_xpts_extra, home_ypts_extra, color="red", s=7)

        away_xpts, away_ypts = [], []
        away_xpts_extra, away_ypts_extra = [], []
        home_xpts_build, home_ypts_build = [], []
        for turn_idx, (launch_counts, launch_plans) in enumerate(zip(self.home_launch_counts, self.home_launch_plans)):
            for launch_count, launch_plan in zip(launch_counts, launch_plans):
                _, _, is_cyclic = extract_flight_plan(0,0,0,launch_plan)
                if "C" in launch_plan:
                    away_xpts_extra.append(turn_idx)
                    away_ypts_extra.append(-launch_count)
                    continue
                away_xpts.append(turn_idx)
                away_ypts.append(-launch_count)
        plt.scatter(away_xpts, away_ypts, color="red", s=4, label=self.away_agent)        
        plt.scatter(away_xpts_extra, away_ypts_extra, color="blue", s=7)

        plt.title("Launch sizes over time")
        plt.xlim(-20,400+20)
        plt.yscale('symlog', linthresh=9)
        plt.yticks([-x for x in limits[2:]] + limits[2:], [-x for x in limits[2:]] + limits[2:])
        plt.legend()
        plt.show()

    def plot_statistics_launch_plan_shapes(self):
        plt.figure(figsize=(15,3))
        xpts = []
        ypts = []
        
        for turn_idx, launch_plans in enumerate(kore_match.home_launch_plans):
            for launch_plan in launch_plans:
                plan_class, target_x, target_y, polarity, construct = fleetplan_matching(launch_plan)
                xpts.append(turn_idx)
                ypts.append(int(plan_class) + np.random.randn()/10)

        plt.scatter(xpts, ypts, color="blue", s=4, label=self.home_agent)        

        xpts = []
        ypts = []
        
        for turn_idx, launch_plans in enumerate(kore_match.away_launch_plans):
            for launch_plan in launch_plans:
                plan_class, target_x, target_y, polarity, construct = fleetplan_matching(launch_plan)
                xpts.append(turn_idx)
                ypts.append(int(plan_class) + np.random.randn()/10)

        plt.scatter(xpts, ypts, color="red", s=4, label=self.away_agent)        

        plt.title("Distribution of flight plans classes over time")
        plt.xlim(-20,400+20)
        plt.yticks([e.value for e in FlightPlanClass], [e.name for e in FlightPlanClass])        
        plt.legend()
        plt.show()

In [ ]:
kore_match = KoreMatch(env.steps)

In [ ]:
kore_match.home_actions[178]  # name-player: instruction_shipcount(_flightplan)

In [ ]:
kore_match.home_kore_stored[178]

In [ ]:
kore_match.away_shipyards[178]  # name-player: location, ship count, turn existence

In [ ]:
kore_match.home_fleets[178]  # name-player: location, kore carried, ship count, direction?, remaining flight plan

# Statisical Visualizations

In [ ]:
kore_match.plot_statistics_kore()

In [ ]:
kore_match.plot_statistics_shipyards()

In [ ]:
kore_match.plot_statistics_ships()

In [ ]:
kore_match.plot_statistics_kore_rates()

In [ ]:
kore_match.plot_statistics_combat_diffs()

In [ ]:
kore_match.plot_statistics_asset_sums()

In [ ]:
kore_match.plot_statistics_launch_sizes()

In [ ]:
kore_match.plot_statistics_launch_plan_shapes()

# Animation Generation

In [ ]:
%%writefile_and_run -a kore_analysis.py

def draw_fleet(x,y,dir_idx,ships_size,kore_amount,color):
    mx,my = x+0.5, y+0.5
    
    icon_size = 0.4
    tip = (0, icon_size)
    left_wing = (icon_size/1.5, -icon_size)
    right_wing = (-icon_size/1.5, -icon_size)
    
    polygon = plt.Polygon([tip, left_wing, right_wing], color=color, alpha=0.3)
    transform = matplotlib.transforms.Affine2D().rotate_deg(270*dir_idx).translate(mx,my)
    polygon.set_transform(transform + plt.gca().transData)
    plt.gca().add_patch(
        polygon
    )
    
    text = plt.text(x+0.1, y+0.75, ships_size, color="purple",
                    horizontalalignment='left', verticalalignment='center')
    text.set_path_effects([matplotlib.patheffects.withStroke(linewidth=2, foreground='w', alpha=0.8)])
    
    kore_amount = int(kore_amount)
    if kore_amount > 0:
        text = plt.text(x+0.1, y+0.25, kore_amount, color="grey",
                        horizontalalignment='left', verticalalignment='center')
        text.set_path_effects([matplotlib.patheffects.withStroke(linewidth=3, foreground='w', alpha=0.8)])

def draw_flight_plan(x,y,dir_idx,plan,fleetsize,color):

    path, construct, is_cyclic = extract_flight_plan(x,y,dir_idx,plan)

    px = np.array([x for x,y in path]) + 0.5
    py = np.array([y for x,y in path]) + 0.5
    for ox in [-21,0,21]:
        for oy in [-21,0,21]:
            plt.plot(px+ox, py+oy, color=color, lw=np.log(fleetsize)**2/1.5, alpha=0.3, solid_capstyle='round')
    for x,y in construct:
        plt.scatter(x+0.5, y+0.5, s=100, marker="x", color=color)

def existence_to_production_capacity(existence):
    if existence >= 294: return 10
    if existence >= 212: return 9
    if existence >= 147: return 8
    if existence >= 97: return 7
    if existence >= 60: return 6
    if existence >= 34: return 5
    if existence >= 17: return 4
    if existence >= 7: return 3
    if existence >= 2: return 2
    return 1
            
def draw_shipyard(x,y,ships_size,existence,color):
    plt.text(x+0.5,y+0.5,"⊕", fontsize=23, color=color,
             horizontalalignment='center', verticalalignment='center', alpha=0.5)
    if ships_size > 0:
        text = plt.text(x+0.1, y+0.75, ships_size, color="purple",
                        horizontalalignment='left', verticalalignment='center')
        text.set_path_effects([matplotlib.patheffects.withStroke(linewidth=2, foreground='w', alpha=0.8)])
    text = plt.text(x+0.9, y+0.25, existence_to_production_capacity(existence), color="black",
                    horizontalalignment='right', verticalalignment='center')
    text.set_path_effects([matplotlib.patheffects.withStroke(linewidth=2, foreground='w', alpha=0.8)])

def draw_kore_amounts(kore_amounts, excluded_xys={}):
    for loc_idx,kore_amount in enumerate(kore_amounts):
        x,y = kaggle_environments.helpers.Point.from_index(loc_idx, 21)
        color = "gainsboro"
        if kore_amount >= 20: color = "silver"
        if kore_amount >= 100: color = "gray"
        if kore_amount >= 500: color = "black"
        if (x,y) not in excluded_xys and kore_amount > 0:
            text = plt.text(x, y, int(kore_amount), color=color, fontsize=7,
                            horizontalalignment='center', verticalalignment='center')
            text.set_path_effects([matplotlib.patheffects.withStroke(linewidth=3, foreground='w', alpha=0.8)])

def draw_statistics(turn_num, home_stored_kore, away_stored_kore):
    plt.text(0-0.5, 21-0.5, f"Kore: {home_stored_kore:.0f}" , color="blue",
             horizontalalignment='left', verticalalignment='bottom')
    plt.text(21-0.5, 21-0.5, f"Kore: {away_stored_kore:.0f}" , color="red",
             horizontalalignment='right', verticalalignment='bottom')
    plt.text(21/2-0.5, 21-0.5, f"Turn: {turn_num:.0f}" , color="black",
             horizontalalignment='center', verticalalignment='bottom')    

In [ ]:
%%writefile_and_run -a kore_analysis.py

def render_turn(self, turn):
    res = self.match_info
    # -0.5 for all x,y for all functions called here as a stopgap to shift axes
    turn_info = res[turn][0]["observation"]
    
    plt.gca().cla()
    plt.gcf().clf()
    plt.gcf().subplots_adjust(left=0.05, bottom=0.05, right=0.95, top=0.95, wspace=None, hspace=None)
    excluded_xys = set()

    color="blue"
    player_idx=0

    for shipyard_info in turn_info["players"][player_idx][1].values():
        loc_idx, ships_count, existence = shipyard_info
        x,y = kaggle_environments.helpers.Point.from_index(loc_idx, 21)
        draw_shipyard(x-0.5,y-0.5, ships_count, existence, color)
        excluded_xys.add((x,y))

    for fleet_info in turn_info["players"][player_idx][2].values():
        loc_idx, kore_amount, ships_size, dir_idx, flight_plan = fleet_info
        x,y = kaggle_environments.helpers.Point.from_index(loc_idx, 21)
        draw_fleet(x-0.5,y-0.5,dir_idx,ships_size,kore_amount, color)
        draw_flight_plan(x-0.5,y-0.5,dir_idx,flight_plan,ships_size, color)
        excluded_xys.add((x,y))

    color="red"
    player_idx=1

    for shipyard_info in turn_info["players"][player_idx][1].values():
        loc_idx, ships_count, existence = shipyard_info
        x,y = kaggle_environments.helpers.Point.from_index(loc_idx, 21)
        draw_shipyard(x-0.5,y-0.5, ships_count, existence, color)
        excluded_xys.add((x,y))

    for fleet_info in turn_info["players"][player_idx][2].values():
        loc_idx, kore_amount, ships_size, dir_idx, flight_plan = fleet_info
        x,y = kaggle_environments.helpers.Point.from_index(loc_idx, 21)
        draw_fleet(x-0.5,y-0.5,dir_idx,ships_size,kore_amount, color)
        draw_flight_plan(x-0.5,y-0.5,dir_idx,flight_plan,ships_size, color)
        excluded_xys.add((x,y))

    draw_kore_amounts(turn_info["kore"], excluded_xys=excluded_xys)
    draw_statistics(turn, self.home_kore_stored[turn], self.away_kore_stored[turn])

    plt.gca().set_xlim(0-0.5,21-0.5)
    plt.gca().set_ylim(0-0.5,21-0.5)
    plt.gca().set_aspect('equal')
    plt.gca().xaxis.set_major_locator(matplotlib.ticker.MaxNLocator(integer=True))
    plt.gca().yaxis.set_major_locator(matplotlib.ticker.MaxNLocator(integer=True))
    
    if self.save_animation:
        plt.savefig(f"frames/{turn:03}.png")

# https://stackoverflow.com/a/24865663/5894029
KoreMatch.render_turn = lambda self, turn: render_turn(self, turn)

In [ ]:
# kore_match.render_turn(200)

In [ ]:
%%writefile_and_run -a kore_analysis.py

def animate(self):
    if self.save_animation:
        os.system("mkdir -p frames")

    self.anim = matplotlib.animation.FuncAnimation(plt.gcf(), self.render_turn, frames=len(self.match_info))
    self.html_animation = IPython.display.HTML(self.anim.to_jshtml())
    plt.close()
    
    if self.save_animation:
        os.system("convert -resize 75% -loop 0 frames/*.png gameplay.gif")
        os.system("rm -rf frames/*.png")

KoreMatch.animate = lambda self: animate(self)

# Function Export

In [ ]:
%reset -sf

# you can generate the animation anywhere given the env object
import pickle
with open('env.pickle', 'rb') as handle:
    env = pickle.load(handle)

In [ ]:
# just import the the KoreMatch class
# kore_analysis might be a different directory, recommend to copy to local directory
from kore_analysis import KoreMatch

kore_match = KoreMatch(env.steps, save_animation=True)
kore_match.animate()

# Strategy Visualizations

In [ ]:
kore_match.html_animation